## 두 번째,
## Yahho Finance 크롤러를 만듭니다.
#### 저는 주식에서 PSR과 매출 성장률을 중요하게 여깁니다. 이러한 데이터는 종목에 들어가 Financials 카테고리에 있습니다.
#### 그래서 저는 이곳의 데이터를 가져오는 크롤러를 만들었습니다.

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np

In [2]:
import time

In [3]:
def make_ticker_rate_df(tickers):
    driver = webdriver.Chrome('./chromedriver')
    ticker_list = list()
    ann_growth_rate_list = list()
    year_list = list()
    recent_year = list()
    total_rev_ttm = list()
    check_number = 0
    for ticker in tickers:
        
        check_number += 1
        print(ticker + ', ' +str(check_number) + '번째 수행 중')
        driver.get('https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        years = list()
        revenues = list()
        for i in range(36,76,2):
            data = soup.find_all(['span'],attrs={'data-reactid':i})
            if data:
                if 36 <= i < 58:
                    # 첫 번째에서 볼 수 있듯이 태그에서 하나의 데이터데이터가 태그의 맨 처음에만 있는 것이 아닙니다. 
                    # 그래서 태그에 있는 모든 데이터를 확인하기 위해 for문을 사용합니다.
                    for i in range(len(data)):
                        if data[i].get_text()[-4:-2] == '20' and data[i].get_text()[-1].isdigit():
                            years.append(data[i].get_text()[-4:])
                            break
                else:
                    data = data[0].get_text()
                    if data and not data[0].isalpha():
                        data = data.replace(',','')
                        revenues.append(data)
                # Yahoo Finance에는 안티 크롤러가 있습니다. 그래서 안티 크롤러를 피해가 위해 한 페이지마다 의도적으로 5초 지연을 시킵니다.
                time.sleep(5)
        if revenues:
            ticker_list.append(ticker)
            ann_growth_rate_list.append(round(((int(revenues[1]) / int(revenues[-1])) ** (1.0/(len(years)-1)) - 1) * 100,2))
            year_list.append(len(years))
            recent_year.append(int(years[0]))
            total_rev_ttm.append(int(revenues[0]))
        else:
            ticker_list.append(ticker)
            ann_growth_rate_list.append(0)
            year_list.append(0)
            recent_year.append(0)
            total_rev_ttm.append(0)          
    print('크롤링 종료')
    driver.close()
    return pd.DataFrame({'ticker':ticker_list,'ann_growth_rate':ann_growth_rate_list,'total_rev_ttm':total_rev_ttm, 'num_of_year':year_list,'recent_year':recent_year})

In [4]:
# 잘 작동하는지 확인을 해봅니다.
tickers = ['GOOG','BABA','NFLX']
make_ticker_rate_df(tickers)

GOOG, 1번째 수행 중
BABA, 2번째 수행 중
NFLX, 3번째 수행 중
크롤링 종료


,ticker,ann_growth_rate,total_rev_ttm,num_of_year,recent_year
0,GOOG,20.83,166030000,3,2019
1,BABA,47.67,584580000,4,2020
2,NFLX,28.82,24996056,4,2020


In [6]:
# S&P500 종목의 태그를 가져옵니다.
import FinanceDataReader as fdr

In [7]:
df_spx = fdr.StockListing('S&P500')
df_spx.head()

,Symbol,Name,Sector,Industry
0,MMM,3M Company,Industrials,Industrial Conglomerates
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services


In [8]:
df_spx_ticker = list(df_spx.Symbol)
df_spx_ticker[:50]

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET']

In [12]:
df_spx_ticker = list(df_spx.Symbol)

### 우선 50개에 대해 먼저 실행해봅니다

In [17]:
start = time.time()
df1 = make_ticker_rate_df(df_spx_ticker[:50])
df1.to_csv('0129rs&p500_rate-50.csv')
print("time :", time.time() - start)

MMM, 1번째 수행 중
ABT, 2번째 수행 중
ABBV, 3번째 수행 중
ABMD, 4번째 수행 중
ACN, 5번째 수행 중
ATVI, 6번째 수행 중
ADBE, 7번째 수행 중
AMD, 8번째 수행 중
AAP, 9번째 수행 중
AES, 10번째 수행 중
AFL, 11번째 수행 중
A, 12번째 수행 중
APD, 13번째 수행 중
AKAM, 14번째 수행 중
ALK, 15번째 수행 중
ALB, 16번째 수행 중
ARE, 17번째 수행 중
ALXN, 18번째 수행 중
ALGN, 19번째 수행 중
ALLE, 20번째 수행 중
LNT, 21번째 수행 중
ALL, 22번째 수행 중
GOOGL, 23번째 수행 중
GOOG, 24번째 수행 중
MO, 25번째 수행 중
AMZN, 26번째 수행 중
AMCR, 27번째 수행 중
AEE, 28번째 수행 중
AAL, 29번째 수행 중
AEP, 30번째 수행 중
AXP, 31번째 수행 중
AIG, 32번째 수행 중
AMT, 33번째 수행 중
AWK, 34번째 수행 중
AMP, 35번째 수행 중
ABC, 36번째 수행 중
AME, 37번째 수행 중
AMGN, 38번째 수행 중
APH, 39번째 수행 중
ADI, 40번째 수행 중
ANSS, 41번째 수행 중
ANTM, 42번째 수행 중
AON, 43번째 수행 중
AOS, 44번째 수행 중
APA, 45번째 수행 중
AAPL, 46번째 수행 중
AMAT, 47번째 수행 중
APTV, 48번째 수행 중
ADM, 49번째 수행 중
ANET, 50번째 수행 중
크롤링 종료
time : 3188.1009726524353


In [13]:
for i in range(50,len(df_spx_ticker),50):
    start = time.time()
    df1 = make_ticker_rate_df(df_spx_ticker[i:i+50])
    file_name = '0129rs&p500_rate-' + str(i+50) + '.csv'
    df1.to_csv(file_name)
    print("time :", time.time() - start)

AJG, 1번째 수행 중
AIZ, 2번째 수행 중
T, 3번째 수행 중
ATO, 4번째 수행 중
ADSK, 5번째 수행 중
ADP, 6번째 수행 중
AZO, 7번째 수행 중
AVB, 8번째 수행 중
AVY, 9번째 수행 중
BKR, 10번째 수행 중
BLL, 11번째 수행 중
BAC, 12번째 수행 중
BK, 13번째 수행 중
BAX, 14번째 수행 중
BDX, 15번째 수행 중
BRKB, 16번째 수행 중
BBY, 17번째 수행 중
BIO, 18번째 수행 중
BIIB, 19번째 수행 중
BLK, 20번째 수행 중
BA, 21번째 수행 중
BKNG, 22번째 수행 중
BWA, 23번째 수행 중
BXP, 24번째 수행 중
BSX, 25번째 수행 중
BMY, 26번째 수행 중
AVGO, 27번째 수행 중
BR, 28번째 수행 중
BFB, 29번째 수행 중
CHRW, 30번째 수행 중
COG, 31번째 수행 중
CDNS, 32번째 수행 중
CPB, 33번째 수행 중
COF, 34번째 수행 중
CAH, 35번째 수행 중
KMX, 36번째 수행 중
CCL, 37번째 수행 중
CARR, 38번째 수행 중
CTLT, 39번째 수행 중
CAT, 40번째 수행 중
CBOE, 41번째 수행 중
CBRE, 42번째 수행 중
CDW, 43번째 수행 중
CE, 44번째 수행 중
CNC, 45번째 수행 중
CNP, 46번째 수행 중
CERN, 47번째 수행 중
CF, 48번째 수행 중
SCHW, 49번째 수행 중
CHTR, 50번째 수행 중
크롤링 종료
time : 3636.4770741462708
CVX, 1번째 수행 중
CMG, 2번째 수행 중
CB, 3번째 수행 중
CHD, 4번째 수행 중
CI, 5번째 수행 중
CINF, 6번째 수행 중
CTAS, 7번째 수행 중
CSCO, 8번째 수행 중
C, 9번째 수행 중
CFG, 10번째 수행 중
CTXS, 11번째 수행 중
CLX, 12번째 수행 중
CME, 13번째 수행 중
CMS, 14번째 수행 중
KO, 15번째 수행 중
CTSH

KeyboardInterrupt: 

#### 보시다시피 시간이 너무 많이 소요됩니다. 한 페이지당 1분이 넘게 소요되죠.
#### 그래서 한 번 크롤러가 실패했을 때, 부담이 큽니다.
#### 이를 보완하기 위해 50개 단위로 for문을 작성해서 csv파일에 저장합니다.

In [13]:
# 350 이후부터
for i in range(350,len(df_spx_ticker),50):
    start = time.time()
    if i >= 500:
        df1 = make_ticker_rate_df(df_spx_ticker[i:-1])
    else:
        df1 = make_ticker_rate_df(df_spx_ticker[i:i+50])
    file_name = '0129rs&p500_rate-' + str(i+50) + '.csv'
    df1.to_csv(file_name)
    print("time :", time.time() - start)

ODFL, 1번째 수행 중
OMC, 2번째 수행 중
OKE, 3번째 수행 중
ORCL, 4번째 수행 중
OTIS, 5번째 수행 중
PCAR, 6번째 수행 중
PKG, 7번째 수행 중
PH, 8번째 수행 중
PAYX, 9번째 수행 중
PAYC, 10번째 수행 중
PYPL, 11번째 수행 중
PNR, 12번째 수행 중
PBCT, 13번째 수행 중
PEP, 14번째 수행 중
PKI, 15번째 수행 중
PRGO, 16번째 수행 중
PFE, 17번째 수행 중
PM, 18번째 수행 중
PSX, 19번째 수행 중
PNW, 20번째 수행 중
PXD, 21번째 수행 중
PNC, 22번째 수행 중
POOL, 23번째 수행 중
PPG, 24번째 수행 중
PPL, 25번째 수행 중
PFG, 26번째 수행 중
PG, 27번째 수행 중
PGR, 28번째 수행 중
PLD, 29번째 수행 중
PRU, 30번째 수행 중
PEG, 31번째 수행 중
PSA, 32번째 수행 중
PHM, 33번째 수행 중
PVH, 34번째 수행 중
QRVO, 35번째 수행 중
PWR, 36번째 수행 중
QCOM, 37번째 수행 중
DGX, 38번째 수행 중
RL, 39번째 수행 중
RJF, 40번째 수행 중
RTX, 41번째 수행 중
O, 42번째 수행 중
REG, 43번째 수행 중
REGN, 44번째 수행 중
RF, 45번째 수행 중
RSG, 46번째 수행 중
RMD, 47번째 수행 중
RHI, 48번째 수행 중
ROK, 49번째 수행 중
ROL, 50번째 수행 중
크롤링 종료
time : 3359.3175961971283
ROP, 1번째 수행 중
ROST, 2번째 수행 중
RCL, 3번째 수행 중
SPGI, 4번째 수행 중
CRM, 5번째 수행 중
SBAC, 6번째 수행 중
SLB, 7번째 수행 중
STX, 8번째 수행 중
SEE, 9번째 수행 중
SRE, 10번째 수행 중
NOW, 11번째 수행 중
SHW, 12번째 수행 중
SPG, 13번째 수행 중
SWKS, 14번째 수행 중
SLG, 15번째 수행 중
